In [0]:
# Databricks notebook source
# ==========================================================
# EXECUTE JOB DDL - Proyecto Liga 1 Perú
# Recibe nombre de archivo DDL y lo ejecuta
# ==========================================================

from env_setup import *
from utils_liga1 import get_dbutils
import os
import re


spark = SparkSession.builder.getOrCreate()
dbutils = get_dbutils()

dbutils dbutils.widgets.text("prm_name_ddl", "")
prm_name_ddl= dbutils.widgets.get("prm_name_ddl") 

print(f"Archivo DDL recibido: {prm_name_ddl}")

container_name = dbutils.secrets.get(scope="secretliga1", key="filesystemname")
adls_account_name = dbutils.secrets.get(scope="secretliga1", key="storageaccount")
catalog_name = "adbliga1futbol" 


In [0]:
# Celda 2: Función para ejecutar TABLA y VISTA en una sola ejecución
def ejecutar_tabla_y_vista(nombre_base, container, storage_account, catalog):
    """
    Ejecuta ambos: tabla y vista en una sola ejecución
    nombre_base: "ddl_md_catalogo_equipos" (sin _view)
    """
    archivos_a_ejecutar = []
    
    # 1. Determinar archivos a ejecutar
    archivo_tabla = f"{nombre_base}.sql"
    archivo_vista = f"{nombre_base}_view.sql"
    
    # Verificar si existen ambos archivos
    carpeta_base = nombre_base  # misma carpeta para tabla y vista
    ruta_tabla = get_workspace_path(f"ddl_deploy/{carpeta_base}/{archivo_tabla}")
    ruta_vista = get_workspace_path(f"ddl_deploy/{carpeta_base}/{archivo_vista}")
    
    if os.path.exists(ruta_tabla):
        archivos_a_ejecutar.append(("TABLA", ruta_tabla))
        print(f"Encontrado archivo de TABLA: {archivo_tabla}")
    else:
        print(f"No se encontró archivo de TABLA: {archivo_tabla}")
    
    if os.path.exists(ruta_vista):
        archivos_a_ejecutar.append(("VISTA", ruta_vista))
        print(f"Encontrado archivo de VISTA: {archivo_vista}")
    else:
        print(f"No se encontró archivo de VISTA: {archivo_vista}")
    
    if not archivos_a_ejecutar:
        raise FileNotFoundError(f"No se encontraron archivos para: {nombre_base}")
    
    # 2. Ejecutar en orden: primero tabla, luego vista
    for tipo, ruta_archivo in archivos_a_ejecutar:
        print(f"\n EJECUTANDO {tipo}: {os.path.basename(ruta_archivo)}")
        print("=" * 60)
        
        # Leer archivo
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            contenido = archivo.read()
        
        # Reemplazar variables
        contenido_procesado = contenido.replace("${container_name}", container)
        contenido_procesado = contenido_procesado.replace("${storage_account}", storage_account)
        contenido_procesado = contenido_procesado.replace("${catalog_name}", catalog)
        
        # Ejecutar SQL
        ejecutar_sql_dividido(contenido_procesado)
        
        print(f"{tipo} ejecutada exitosamente")

In [0]:
# Celda 3: Función para ejecutar SQL dividido
def ejecutar_sql_dividido(sql_content):
    """
    Divide el SQL en statements y los ejecuta uno por uno
    """
    # Dividir por punto y coma
    statements = []
    statement_actual = ""
    
    for linea in sql_content.split('\n'):
        linea_limpia = linea.strip()
        
        # Saltar líneas vacías o comentarios
        if not linea_limpia or linea_limpia.startswith('--'):
            continue
            
        statement_actual += linea + "\n"
        
        # Si termina con punto y coma, es un statement completo
        if linea.rstrip().endswith(';'):
            statements.append(statement_actual.strip())
            statement_actual = ""
    
    # Si queda algo sin punto y coma, agregarlo
    if statement_actual.strip():
        statements.append(statement_actual.strip())
    
    print(f"Encontrados {len(statements)} statements SQL")
    
    # Ejecutar cada statement
    for i, statement in enumerate(statements, 1):
        print(f"\nEjecutando statement {i}/{len(statements)}:")
        print("=" * 50)
        
        # Mostrar primeras líneas del statement
        lineas = statement.split('\n')
        for j, linea in enumerate(lineas[:3]):  # Mostrar primeras 3 líneas
            print(f"   {linea}")
        if len(lineas) > 3:
            print("   ...")
            
        print("=" * 50)
        
        try:
            spark.sql(statement)
            print(f"Statement {i} ejecutado exitosamente")
        except Exception as e:
            print(f"Error en statement {i}: {e}")
            raise

In [0]:
# COMMAND ----------
# Celda 4: Ejecución principal CORREGIDA
try:
    print("\n INICIANDO EJECUCIÓN DDL AUTOMÁTICA...")
    print("=" * 60)
    print(f" Parámetro recibido: {prm_name_ddl}")
    
    # Validar parámetro obligatorio
    if not prm_name_ddl or not prm_name_ddl.strip():
        raise ValueError("El parámetro 'prm_name_ddl' está VACÍO")
    
    # Ejecutar TABLA y VISTA automáticamente
    ejecutar_tabla_y_vista(
        prm_name_ddl,  # "ddl_md_catalogo_equipos"
        container_name, 
        adls_account_name, 
        catalog_name
    )
    
    print("=" * 60)
    print(f"DDL COMPLETADO: Tabla y Vista creadas para {prm_name_ddl}")
    
    # SOLO para éxito - usar notebook.exit()
    dbutils.notebook.exit("SUCCESS")
    
except Exception as e:
    print("=" * 60)
    print(f"ERROR CRÍTICO: {str(e)}")
    
    # PARA ERRORES - lanzar excepción (NO usar notebook.exit())
    # Esto hará que el job marque como FAILED en Databricks
    raise Exception(f"Fallo en ejecución DDL: {str(e)}")